<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_text_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

In [2]:
%%time

!git clone -l -s git://github.com/kavyajeetbora/EAST.git cloned-repo
%cd cloned-repo
!ls

fatal: destination path 'cloned-repo' already exists and is not an empty directory.
/content/cloned-repo
cloned-repo	     EAST_dataset.ipynb        model.py
dataset_original.py  EAST_text_detector.ipynb  __pycache__
dataset.py	     loss.py		       train.py
CPU times: user 112 ms, sys: 18.9 ms, total: 131 ms
Wall time: 8.04 s


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from dataset_original import custom_dataset

# AI4Bharat dataset
image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/Simple YOLO model/Data/Sample Images'
label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/Simple YOLO model/Data/Sample Annotations'


# icdar dataset
# image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Images'
# label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Labels'

training_dataset = custom_dataset(image_address, label_address)
print(len(training_dataset))

428


In [0]:
def extract_vertices(lines):
	'''extract vertices info from txt lines
	Input:
		lines   : list of string info
	Output:
		vertices: vertices of text regions <numpy.ndarray, (n,8)>
		labels  : 1->valid, 0->ignore, <numpy.ndarray, (n,)>
	'''
	labels = []
	vertices = []
	for line in lines:
		vertices.append(list(map(int,line.rstrip('\n').lstrip('\ufeff').split(',')[:8])))
		label = 0 if '##::' in line else 1
		labels.append(label)
	return np.array(vertices), np.array(labels)

In [0]:
from dataset_original import *

In [7]:
img = Image.open(image_address+'/0.jpg')

with open(label_address+'/0.txt', 'r') as f:
			lines = f.readlines()
vertices, labels = extract_vertices(lines)

img, vertices = adjust_height(img, vertices) 
img, vertices = rotate_img(img, vertices)
img, vertices = crop_img(img, vertices, labels, 512)

for vertice in vertices:
  print(vertice)
  print('-'*20)
  
print(labels)

[ 82.91349255 346.64709295 231.36786531 138.71850134 156.14622678
 246.83744092 233.39168583 300.70894597]
--------------------
[274.82842929 520.27214027 431.46857415 313.34011032 163.08840631
 254.03508938 241.69253536 318.82236356]
--------------------
[461.94313935 622.94679771 563.10185924 488.42202902 164.08074896
 253.91157393 244.79729777 311.3819032 ]
--------------------
[1 1 1]


In [8]:
score_map, geo_map, ignored_map = get_score_geo(img, vertices, labels, 0.25, 512)

torch.sum(score_map)

tensor(685.)

## Training

In [9]:
%%time
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=8)

imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))

print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())

print(torch.sum(gt_score))

torch.Size([8, 3, 512, 512]) torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128]) torch.Size([8, 1, 128, 128])
tensor(10110.)
CPU times: user 5.59 s, sys: 4.79 s, total: 10.4 s
Wall time: 4.77 s


In [0]:
from model import EAST

model = EAST()

In [11]:
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128])


In [12]:
from loss import Loss

loss_fn = Loss()

loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.86142826, angle loss is 0.91079426, iou loss is 1.70341551


tensor(11.6728, grad_fn=<AddBackward0>)

## Training Full Batch

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[5//2], gamma=0.1)
criterion = Loss()

In [0]:
import os
import time

def train(train_img_path, train_gt_path, pths_path, batch_size, lr, num_workers, epoch_iter, interval):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = EAST()
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[epoch_iter//2], gamma=0.1)
  
    for epoch in range(epoch_iter):
      model.train()
      scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      if (epoch + 1) % interval == 0:
        state_dict = model.module.state_dict() if data_parallel else model.state_dict()
        torch.save(state_dict, os.path.join(pths_path, 'model_epoch_{}.pth'.format(epoch+1)))

In [16]:
training = train(image_address, label_address, 'models', batch_size=8, lr=1e-3, num_workers=0, epoch_iter=1, interval=5)

428


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


classify loss is 0.88356185, angle loss is 0.99434370, iou loss is 1.45743048
Epoch is [1/1], mini-batch is [1/53], time consumption is 0.87976980, batch_loss is 12.28442955
classify loss is 0.90996081, angle loss is 0.87531251, iou loss is 1.50867319
Epoch is [1/1], mini-batch is [2/53], time consumption is 0.84924746, batch_loss is 11.17175961
classify loss is 0.91342682, angle loss is 0.84967631, iou loss is 1.58364463
Epoch is [1/1], mini-batch is [3/53], time consumption is 0.80443692, batch_loss is 10.99383450
classify loss is 0.89080280, angle loss is 0.81135327, iou loss is 1.56056738
Epoch is [1/1], mini-batch is [4/53], time consumption is 0.80710411, batch_loss is 10.56490326
classify loss is 0.91931546, angle loss is 0.92152983, iou loss is 1.68006837
Epoch is [1/1], mini-batch is [5/53], time consumption is 0.81151056, batch_loss is 11.81468201
classify loss is 0.90065616, angle loss is 0.80441743, iou loss is 1.71408582
Epoch is [1/1], mini-batch is [6/53], time consumpti

5.8